In [231]:
!pip install openai==1.2.0
import json

In [232]:
from openai import OpenAI # openai==1.2.0

client = OpenAI(
    api_key="up_TQpptc9W6GuPYyQ1o2rIYpgMiY6UN",
    base_url="https://api.upstage.ai/v1/solar"
)

In [233]:
messages = [
    # 메세지 로그
    {
        "role": "system",
        "content": "너는 숙박 업소 챗봇이야. 고객들에게 젠틀하게 서비스를 제공하도록 해."
    }
]

In [234]:
formatter_log = [
    # 포맷 로그
    {
        "role": "system",
        "content": "너는 숙박 업소 챗봇이야. 고객들들에게 최선을 다해 서비스를 제공하도록 해. 너는 내가 주는 정보를 고객들에게 잘 표현하여 전달하는 역할이야. 내가 주는 딱딱한 정보들을 고객들이 만족할 수는 서비스로 표현을 만들어서 제공하도록 해."
    }


]

In [235]:
# Function Calling을 위한 함수들 목록
tools = [
        {
            "type": "function",
            "function": {
                "name": "use_data",
                "description": "고객이 숙박시설에 대한 분위기 등의 소개를 물어보면 '기본정보', 숙박시설의 위치를 물어보면 '위치정보', 숙박시설 내부 시설 내용을 물어보면 '시설정보', 예약에 대한 내용을 물어보면 '예약정보', 숙박시설에서 제공하고 있는 서비스에 대해 물어보면 '서비스정보', 식사메뉴와 같이 때에 따라 달라질 수 있는 정보들을 물어보면 '가변정보'로 분류해줘. 숙박업소 챗봇 기준으로 해당 분류에 적합하지 않다고 생각하는 것은 분류에 넣지마.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "category": {
                            "type": "string",
                            "description": "기본정보, 위치정보, 시설정보, 예약정보, 서비스정보, 가변정보",
                        },
                        "category": {"type": "string", "enum": ["기본정보", "위치정보", "시설정보", "예약정보", "서비스정보", "가변정보"]},
                    },
                    "required": ["category"],
                },
            },
        }


    ]

In [236]:
def formatter(info):

  formatter_log.append({
        "role": "user",
        "content": info
    })

  response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=formatter_log,
        stream=False

  )

  full_response = response.choices[0].message.content

  return full_response  # 최종 응답을 반환

In [237]:
def chat(content):
    # 사용자 메시지를 추가하는 함수
    messages.append({
        "role": "user",
        "content": content
    })

    # 새로운 stream 객체 생성
    response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=messages,
        stream=False,
        tools=tools,
        tool_choice="auto",
    )


    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls


    if tool_calls:
        for tool_call in tool_calls:
            # 인자들 확인
            function_args = json.loads(tool_call.function.arguments)
            selected_category = function_args.get("category")

            # 선택된 category를 기반으로 응답 생성
            tool_response = formatter(f"선택된 카테고리는 '{selected_category}'입니다.")
            messages.append({
                "role": "assistant",
                "content": tool_response,
                "category": selected_category
            })
    else:
        # tool_choice가 없으면 일반적인 모델 응답을 추가
        messages.append({
            "role": "assistant",
            "content": response.choices[0].message.content
        })

    return messages[-1]["content"]



In [238]:
print(chat("주변에 경찰서가 있니?"))

알겠습니다. 이제부터 '위치정보'에 관련된 정보를 제공하거나 질문에 답변해드릴게요. 어떤 도움이 필요하신가요?


In [239]:
formatter_log

[{'role': 'system',
  'content': '너는 숙박 업소 챗봇이야. 고객들들에게 최선을 다해 서비스를 제공하도록 해. 너는 내가 주는 정보를 고객들에게 잘 표현하여 전달하는 역할이야. 내가 주는 딱딱한 정보들을 고객들이 만족할 수는 서비스로 표현을 만들어서 제공하도록 해.'},
 {'role': 'user', 'content': "선택된 카테고리는 '위치정보'입니다."}]

In [240]:
messages

[{'role': 'system', 'content': '너는 숙박 업소 챗봇이야. 고객들에게 젠틀하게 서비스를 제공하도록 해.'},
 {'role': 'user', 'content': '주변에 경찰서가 있니?'},
 {'role': 'assistant',
  'content': "알겠습니다. 이제부터 '위치정보'에 관련된 정보를 제공하거나 질문에 답변해드릴게요. 어떤 도움이 필요하신가요?",
  'category': '위치정보'}]